# Training a Custom Connected Layer with a ResNet50V2 Base

In this notebook, we will transfer learn with a trained ResNetV2 image classifier. Wikipedia defines, transfer learning as an approach which focuses on storing knowledge gained while solving one problem and applying it to a different but relation problem. In this context, the ResNetV2 classifier was trained on the ResNet image dataset. This dataset contains millions of images and hundreds of classes. By using a trained ResNet50V2 model as the starting point for our classifier we can save massive amounts of training time, and likely achieve performance that would be difficult to replicate with just the food-101 dataset. This is because, the ResNet50V2 model will have learned relevants features (e.g., edges, patterns) that require millions of examples to learn (with a traditional CNN approach). 

As a first step, in our transfer learning journey, we will 'lock the convolutional base' of the ResNet50V2 model and only train a custom fully connected top layer. This is critical as training the entire ResNet50V2 model on the food-101 dataset immediately would result in large losses, causing large model weight adjustments, and likely destroy much of the ResNet50V2's ability to detect relevant features in images. In future notebooks, we will unlock part of the ResNet50V2's convolutional base for training, to enable a process called fine tuning.

In [1]:
# Import dependencies
from keras.applications import ResNet50V2
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


# Setting up a preprocessing pipeline

Our primary prepcrocessing step will be image augmentation and normalization. First, we normalize the RGB values to be on a [0,1] scale. This is done to accelerate model training, by reducing computational intensiveness. Next, we apply variants of the image augmentations explored in the prior notebook. 

Finally, the ImageDataGenerator object will split our train dataset into train and validation sets. This is important as the validation set will help us avoid overfitting on the train dataset. Note, we will not use the test dataset until we are confident we have built a well-optimized and high performance classifier. 

Note, we also specify that all of our images will be rescaled to be 512x512 pixels. As shown previously many of the images in our dataset our roughly this size. Many image classifiers work with 256x256 or 512x512 image sizes as their inputs. 

In [2]:
# create the image data generator object
datagen = ImageDataGenerator(rescale=1./255, 
                             brightness_range=[0.8,1.2],
                             rotation_range=30,
                             horizontal_flip=True,
                             zoom_range=[0.8,1.2],
                             fill_mode='nearest',
                             validation_split=0.03)

# prints "XXX images belonging to 101 classes"
train_datagen = datagen.flow_from_directory('data/train/', seed=42, class_mode='categorical', subset='training', target_size=(512,512))
# prints "XXX images belonging to 101 classes"
val_datagen = datagen.flow_from_directory('data/train/', seed=42, class_mode='categorical', subset='validation', target_size=(512,512)) 

# the number of steps per epoch will be an important parameter to specify during model training
train_steps = len(train_datagen) #1894
val_steps = len(val_datagen) #474

# the number of nodes in our output layer must match the number of classes in our dataset
classes = len(list(train_datagen.class_indices.keys())) #101

Found 73528 images belonging to 102 classes.
Found 2222 images belonging to 102 classes.


Import a trained ResNet50V2 model without a fully connected top, specify that pooling layers should use the average (versus max) of each filter, and that the input shape to the model will be a 512x512 image with three channels (RGB).  

In [3]:
conv_base = ResNet50V2(weights='imagenet', include_top=False, pooling='avg', input_shape=(512, 512, 3))

Specify the architecture of our model. As noted, the conv_base will be the ResNet50V2 model. The top hidden layers of our model will be a fully connected layer with 512 notes. We will use batchnormalization and dropout to improve the generalization of our model and avoid overfitting. 

The output layer of the model will be a 101 node connected layer with a softmax activation function. 101 nodes is necessary as we have 101 classes. Softmax is the standard activation function for output layers for multiclass classification problems. 

Finally, we will make the conv_base layers untrainable at this point. Per the above discussion, at this stage we will only train the top layers. 

In [4]:
model = Sequential()

model.add(conv_base)
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(classes, activation='softmax'))

conv_base.trainable = False

Callbacks enable us to monitor training and intervene as defined. At various points during training, callbacks included tensorboards (for monitoring) and learning rate schedules (decrease learning rate as epochs increase); however, ultimately I found that two simply callbacks were most useful. The early stopping callback allows us to halt training if our validation accuracy does not improve for 2 straight epochs. The model checkpoint saves the model after each epoch if validation accuracy exceeds prior epochs. 

In [6]:
callbacks = [
    ModelCheckpoint(
        filepath='trained_top_resnet.h5',
        monitor='val_acc',
        save_best_only=True,
    ),
    EarlyStopping(
        monitor='val_acc',
        patience=2,
    )
]

We compile our model with a standard optimizer, specify the loss as categorical_crossentropy (standard for multiclass classification), and specify the metrics we want to monitor as accuracy and top_k_categorical_accuracy (default k=5). 

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['acc','top_k_categorical_accuracy'])

As seen below, validation_accuracy stops improving at epoch 9. At epoch 9, we achieve a validation accuracy of 44.7%. Certainly not great; however, given that we are not training the convolutional base at all at this point this is an acceptable level of performance. In future notebooks, we will explore the impact fine tuning has on performance. 

In [8]:
history = model.fit_generator(
    train_datagen,
    steps_per_epoch=train_steps,
    epochs=50,
    verbose=2,
    validation_data=val_datagen,
    validation_steps=val_steps,
    callbacks=callbacks
)

Epoch 1/50
 - 5927s - loss: 2.8424 - acc: 0.3389 - top_k_categorical_accuracy: 0.6004 - val_loss: 2.4957 - val_acc: 0.4028 - val_top_k_categorical_accuracy: 0.6962
Epoch 2/50
 - 5420s - loss: 2.3840 - acc: 0.4247 - top_k_categorical_accuracy: 0.6945 - val_loss: 2.7581 - val_acc: 0.4203 - val_top_k_categorical_accuracy: 0.6962
Epoch 3/50
 - 5647s - loss: 2.3327 - acc: 0.4348 - top_k_categorical_accuracy: 0.7059 - val_loss: 3.3645 - val_acc: 0.4334 - val_top_k_categorical_accuracy: 0.7030
Epoch 4/50
 - 5855s - loss: 2.3134 - acc: 0.4382 - top_k_categorical_accuracy: 0.7082 - val_loss: 2.1044 - val_acc: 0.4176 - val_top_k_categorical_accuracy: 0.6962
Epoch 5/50
 - 5889s - loss: 2.3103 - acc: 0.4396 - top_k_categorical_accuracy: 0.7077 - val_loss: 2.0695 - val_acc: 0.4347 - val_top_k_categorical_accuracy: 0.7025
Epoch 6/50
 - 5761s - loss: 2.3109 - acc: 0.4399 - top_k_categorical_accuracy: 0.7086 - val_loss: 2.8691 - val_acc: 0.4401 - val_top_k_categorical_accuracy: 0.7084
Epoch 7/50
 - 57